Uso de un socket UDP
Como TCP, UDP pertenece a la capa de transporte del modelo OSI pero, a diferencia de él, no realiza una conexión previa de sincronización ni
garantiza la correcta recepción de los datos. Por el contrario, integra un sistema que asegura la integridad del dato transmitido, como TCP.

UDP es menos fiable que TCP, puesto que puede perder paquetes, pero es mucho más rápido, pues necesita muchos menos mensajes de ida y
vuelta. Es preferible para tecnologías en las que la fiabilidad no sea una necesidad principal respecto a la rapidez, como en el caso de la voz
sobre IP, el streaming o los juegos en red.

Esto es cierto en la medida en que las aplicaciones correspondientes sean capaces de superar una pequeña pérdida de datos, o bien
reconstruirlos (siempre que sea más rápido que volver a solicitarlos), o bien sustituirlos.

Por otro lado, TCP se utiliza para establecer un diálogo entre un cliente y un servidor, mientras que UDP se utiliza para enviar datos desde un
cliente hacia el servidor sin que el cliente espere una respuesta.

En este caso, se utilizan sockets, pero como los protocolos TCP y UDP difieren en su funcionamiento, esta diferencia es visible en la manera en
que se escriben un servidor y un cliente.
De este modo, el servidor no espera sincronizar ninguna conexión con un cliente y el cliente no se conecta a un servidor antes de enviarle los
datos.

Por lo demás, la manera de gestionar los datos es idéntica a lo que hemos visto para un servidor TCP. Ejemplos más sencillos que los que se
muestran en este libro permiten observar las diferencias entre los protocolos TCP (http://wiki.python.org/moin/TcpCommunication) y UDP
(http://wiki.python.org/moin/UdpCommunication). En ambos casos, los servidores muestran el dato recibido y, para TCP, se confirma y se
devuelve un dato al cliente.

El siguiente ejemplo está en la línea del que hemos visto para TCP: el cliente envía información al servidor. Las líneas inútiles se dejan como
comentario para poner de relieve las diferencias con TCP.


In [ ]:
#!/usr/bin/python3
import socket
params=('127.0.0.1',8808)
BUFFER_SIZE=1024 #default
datos= {}
s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)#SOCK_STREAM
s.bind(params)
#Lassiguienteslíneasnotienensentidoen UDP:
#s.listen(1)
#conn,addr=s.accept()
#print(’Conexiónaceptada:%s’%str(addr))

Se observa claramente, en la primera sección del código, que no es necesario quedarse a la escucha e iniciar una conexión.
En lo que mostramos a continuación, la diferencia de semántica entre recv y recvfrom pone de relieve el hecho de que el
métodorecvfromrecibe a la vez el dato y la dirección del cliente.

In [ ]:
while True:
    #data=conn.recv(BUFFER_SIZE)
    data,addr =s.recvfrom(BUFFER_SIZE)
    print('Conexiónrecibida:%s'%str(addr))
    print('Datorecibido:%s'%data)
    if not data:
        break
    print('Datorecibido:%s'%data)
    try:
        number =int(data)
    except:
        response=b'E'
        phrase ="’%s’noesunnúmeroentero"%data
    else:
        if isprime(number):
            phrase= "’%s’esunnúmeroprimo"%number
            response =b'T'
        else:
            phrase= "’%s’noesunnúmeroprimo"%number
            response =b'F'
    finally:
        print(response)
    #conn.close()

Cabe destacar que, si bien para TCP hacía falta un bucle para gestionar cada conexión y otro para gestionar los intercambios en el seno de la
propia conexión, en UDP basta con un único bucle. Para que el ejemplo pueda terminar correctamente, se agrega un caso de uso particular que
permite salir del bucle.
He aquí el código de cliente correspondiente:

In [ ]:
#!/usr/bin/python3
import socket
params=('127.0.0.1',8808)
BUFFER_SIZE=1024 #default
s=socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
#s.connect(params)

En este caso, no hace falta una conexión. Es en el momento en que se envían los datos cuando se indica la información relativa al servidor al
que van dirigidos.


In [ ]:
cifras=[4j,4,5,-5,17,29,2**50,2**50-1]


Los mensajes enviados son de distinta naturaleza, y el último mensaje permite interrumpir el servidor propiamente dicho. He aquí el bucle
donde se observa una diferencia semántica entre sendy sendto, que se utiliza para poner de relieve la diferencia entre ambos protocolos y,
en consecuencia, los parámetros esperados.


In [ ]:
for cifra in cifras:
    print("Envíodeunmensaje%s"%cifra)
#s.send((’%s\n’%cifra).encode(’utf8’))
s.sendto(('%s\n'%cifra).encode('utf8'),params)
data=s.recv(BUFFER_SIZE)
if len(data)==0:
    print("\tSinrespuesta")
elif data==b'E\n':
    print("\tSehaproducidounerroren elservidor")
elif data==b'T\n':
    print("\tSetratadeunnúmeroprimo")
elif data==b'F\n':
    print("\tElnúmeronoesprimo")
else:
    print("\tEldato recibidonosecomprende:%s" %data)
#s.close()


A continuación hay que ejecutar el servidor, luego el cliente, en dos consolas diferentes para ver el resultado obtenido.

Como puede observarse, salvo ciertos detalles que se explican por la diferencia entre ambos protocolos, buena parte del código es idéntico
entre TCP y UDP. Para ser más precisos, el código de negocio es idéntico.

Por motivos de estandarización, existe una capa ligeramente menos de bajo nivel, que se presenta en las dos secciones siguientes y que
permite ir todavía más allá para unificar el uso de los protocolos de red.


Creación de un servidor UDP

Del mismo modo que existen dos clases TCPServer y StreamRequestHandler para TCP, existen dos clases UDPServeryDatagramRequestHandlerpara UDP, con el mismo objetivo.

La ventaja de utilizar handlers resulta evidente: enmascaran por completo la complejidad de lo que ocurre realmente en el nivel de red para
permitir una codificación similar.

De este modo, si tomamos el código anterior, y remplazamos las siglas TCP por UDP, y la herencia de StreamRequestHandler hacia 
DatagramRequestHandler, el servidor UDP obtenido ofrece exactamente el mismo servicio que el que habíamos escrito para TCP.

El código es similar aunque, no obstante, los procesos implementados son muy diferentes.
Para poner esto de relieve, conviene reutilizar el cliente UDP realizado antes.
Es posible, también, transmitir datos de la misma manera en TCP y en UDP, si bien ambos protocolos tienen objetivos diferentes y modos de
funcionamiento realmente distintos.

En este caso, estamos trabajando en el bajo nivel, aunque ya hemos hecho una buena abstracción en lo relativo a las capas de red y somos
capaces de dar respuesta a muchas problemáticas clásicas. Para utilizar niveles de abstracción más elevados, es necesario utilizar los
frameworks.

Cuando el servidor y el cliente están ubicados en la misma máquina, es una pena perder el tiempo en la capa de red para negociar una
conexión ya que es posible instaurar un diálogo mucho más rápido.

Este es el rol de los sockets IPC o sockets de dominio UNIX. La comunicación tiene lugar directamente en el núcleo y los procesos pueden
intercambiar directamente datos.

Antes de establecer una conexión, es importante comprobar si el archivo utilizado para referenciar el socket no existe:
Una vez realizado, es posible crear el servidor que lo va a utilizar

In [ ]:
import os,os.path
if os.path.exists("/tmp/mycustomsock"):
    os.remove("/tmp/mycustomsock")

In [ ]:
import  socket
server=socket.socket(socket.AF_UNIX,socket.SOCK_DGRAM)
server.bind("/tmp/mycustomsock")

Y el cliente:


In [ ]:
client=socket.socket(socket.AF_UNIX,socket.SOCK_DGRAM)
client.connect("/tmp/mycustomsock")

Para el resto, el funcionamiento es similar al que se ha visto anteriormente.
Cabe destacar que MySQL y PostgreSQL, entre otros, utilizan este sistema y muchos servidores web se privan de esta posibilidad,
relativamente útil.
Las posibilidades ofrecidas por un servidor dependen, también, del sistema operativo. Para que escuche algo distinto al bucle local, deben
utilizarse herramientas suplementarias:

In [ ]:
import os
kernel,hostname,release,version,hardware=os.uname()
print('Hostname:%s' %hostname)


In [ ]:
import socket
address=socket.gethostbyname(hostname)
print(address)
{"127.0.1.1"}
socket.gethostbyaddr(address)
('srv.ejemplo.com',[],['127.0.1.1'])


O directamente

In [ ]:
socket.gethostbyname_ex(hostname)
('srv.ejemplo.com',[],['127.0.1.1'])

A continuación se utilizan las características propias de la creación del servidor TCP o UDP para que el servidor escuche algo más que el bucle
local, es decir, la propia máquina.
Conviene destacar que Python 3.3 introdujo socket.sendmsg, que es un método de más alto nivel que permite enviar datos complejos de
manera sencilla (este método se ha mejorado con Python 3.5). Python 3.5 ha creado también socket.sendfileque evita tener que hacer
unfile.ready un socket.sendy que permite también que la operación completa se realice a nivel del núcleo, con un mejor rendimiento.
